# *Python from Scratch LiveLessons*
## Lesson 7: Pie Factory&#8212;Object-Oriented Python
<img src="misc/pie_cover_variation_.png" width="60%"  />

API is short for Application Program Intersface. This is how we make it so one program talks to another. We will conduct the following steps:

1. Read the API documentation for our *Factory Conveyor Belt* we will use (already exists).
1. Create callbacks.
1. Use a Mock Browser (simulation) to handle running our Factory. 
1. Run our Mock Browser.

In Lesson, 8 we will finalize our whole program by hooking up our code to the the actual web browser. By using this Mock Server we will write the callback functions.  By breaking this up into smaller parts we simplify the approach.  

### 7.1 Read the API documentation for our Factory Conveyor Belt we will use (already exists).

Below is our design of basic flow:

<img src="misc/workflow_2.png" width="60%"  />


The callbacks are functions that carry out the task at hand for each operation. The FactoryConveyorBelt holds the data. So we create an instance of FactoryConveryorBelt first:



Next we create a Pie and add it to the belt:

### 7.2 Creating callbacks

The belt knows about certain methods:

We can add a simple callback `echo()` to all of theses. Please note that all callbacks take the `callback_app` and `message` as arguments:

### 7.3 Using the mock browser


<img src="misc/workflow_2.png" width="60%"  />

Already created for you is the Mock Browser:

### 7.4 Running



### 7.5 Creating more callbacks and Putting it all together

Now we add the callbacks `bake_callback()` and `oven_callback()`: